In [1]:
import pandas as pd
import datetime
import tqdm

In [2]:
df = pd.read_csv('/Users/sava/Desktop/M1/XML_TEI/XML_devoir/fdb_scraped_pulito.csv')

In [6]:
df = df.drop_duplicates(subset=['Titolo','Autore','Testo'])

In [8]:
def thread_to_xml(thread, titolo):
    """
    thread: pandas Dataframe

    returns an xml string
    """

    thread = thread.reset_index(drop=True)

    thread_post_list = ''

    for i in range(len(thread)):
        row = thread.iloc[i]
        date = datetime.datetime.strptime(row['Data'].split('T')[0], '%Y-%m-%d')
        year = date.year
        month = date.month
        day = date.day

        date_str = f"""<date year="{year}" month="{month}" day="{day}">{row['Data'].split('T')[0]}</date>"""
        time = f"<time>{row['Data'].split('T')[1][:-5]}</time>"
        autor_name = row['Autore'].replace(' ','_')
        autore = f"""<author who="#{autor_name}">{row['Autore']}</author>"""
        if type(row['Testo'])==str:
            text = f"<text>{row['Testo'].replace('<', '&lt;').replace('>','&gt;')}</text>"
        else:
            text = f"<text>{row['Testo']}</text>"

        post = f'''<post id="{i}">
        {date_str}
        {time}
        {autore}
        {text}
        </post>'''

        thread_post_list += post+'\n'

        titolo = titolo.replace('"','\'')
        thread_str = f'''<thread title="{titolo}">
        {thread_post_list}</thread>'''
    
    return thread_str
    


In [9]:
xml_string = ''
for titolo in tqdm.tqdm(df['Titolo'].unique()):
    thread_xml = thread_to_xml(df[df['Titolo']==titolo], titolo)
    xml_string += f'{thread_xml}\n'

100%|██████████| 2377/2377 [02:27<00:00, 16.08it/s]


In [10]:
final = f"""<?xml version="1.0" encoding="UTF-8"?>
<forum>
{xml_string}</forum>"""

In [11]:
final = final.replace('&','&amp;')

In [12]:
with open('fdb_xml.xml','w') as file:
    file.write(final)